Praca projektowa nr 4
==================
Mariola Bartosik          
----------------

                                                  


In [1]:
import pandas as pd
import os
import sqlite3
import tempfile

**Otwieramy pliki csv.**

In [2]:
Tags = pd.read_csv("data/Tags.csv")
Badges = pd.read_csv("data/Badges.csv")
Posts = pd.read_csv("data/Posts.csv")
PostLinks = pd.read_csv("data/PostLinks.csv")
Users = pd.read_csv("data/Users.csv")
Votes = pd.read_csv("data/Votes.csv")
Comments = pd.read_csv("data/Comments.csv")

In [3]:
# sciezka dostępu do bazy danych:
base = os.path.join(tempfile.mkdtemp(), 'przyklad.db')
if os.path.isfile(base): # jesli baza już istneje...
    os.remove(base) # ...usuniemy ja

In [4]:
conn = sqlite3.connect(base) # połączenie do bazy danych

In [5]:
Badges.to_sql("Badges", conn) # importujemy ramkę danych do bazy danych
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)
Tags.to_sql("Tags", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)

Zadanie 1
------------


### 1. Rozwiązanie za pomocą wywołania *pandas.read_sql_query("""zapytanie SQL""")*

In [6]:
df_sql_1 = pd.read_sql_query("""
                    SELECT Title, Score, ViewCount, FavoriteCount
                    FROM Posts
                    WHERE PostTypeId=1 AND FavoriteCount >= 25 AND ViewCount>=10000
                    """, conn)
df_sql_1 

,Title,Score,ViewCount,FavoriteCount
0,When traveling to a country with a different c...,136,16838.0,35.0
1,"How can I do a ""broad"" search for flights?",95,33554.0,49.0
2,Tactics to avoid getting harassed by corrupt p...,156,13220.0,42.0
3,How to avoid drinking vodka?,149,15197.0,29.0
4,Flight tickets: buy two weeks before even duri...,109,49440.0,36.0
5,"OK we're all adults here, so really, how on ea...",306,73808.0,79.0
6,Japanese etiquette: Most common (and offensive...,59,24607.0,27.0
7,Why are airline passengers asked to lift up wi...,210,117228.0,34.0
8,How to intentionally get denied entry to the U...,219,63873.0,53.0
9,"When I travel next, I want to reach the point ...",104,10279.0,25.0


### 2. Rozwiązanie za pomocą wywołania ciągu „zwykłych” metod i funkcji z pakietu *pandas*.


In [7]:
df_pd_1 = (Posts[(Posts.PostTypeId == 1) & (Posts.FavoriteCount >= 25) & (Posts.ViewCount >= 10000)]
[['Title', 'Score', 'ViewCount','FavoriteCount']])
df_pd_1.reset_index(drop=True, inplace=True)
df_pd_1

,Title,Score,ViewCount,FavoriteCount
0,When traveling to a country with a different c...,136,16838.0,35.0
1,"How can I do a ""broad"" search for flights?",95,33554.0,49.0
2,Tactics to avoid getting harassed by corrupt p...,156,13220.0,42.0
3,How to avoid drinking vodka?,149,15197.0,29.0
4,Flight tickets: buy two weeks before even duri...,109,49440.0,36.0
5,"OK we're all adults here, so really, how on ea...",306,73808.0,79.0
6,Japanese etiquette: Most common (and offensive...,59,24607.0,27.0
7,Why are airline passengers asked to lift up wi...,210,117228.0,34.0
8,How to intentionally get denied entry to the U...,219,63873.0,53.0
9,"When I travel next, I want to reach the point ...",104,10279.0,25.0


### 3. Sprawdzenie tożsamości zwracanych wyników

In [8]:
df_pd_1.equals(df_sql_1)

True

Zadanie 2
---------

### 1. Rozwiązanie za pomocą wywołania *pandas.read_sql_query("""zapytanie SQL""")*

In [9]:
df_sql_2 = pd.read_sql_query("""
                    SELECT Posts.Title, RelatedTab.NumLinks FROM
                        (SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
                        FROM PostLinks GROUP BY RelatedPostId) AS RelatedTab
                    JOIN Posts ON RelatedTab.PostId=Posts.Id
                    WHERE Posts.PostTypeId=1
                    ORDER BY NumLinks DESC
                    LIMIT 10
                    """, conn)
df_sql_2

,Title,NumLinks
0,Is there a way to find out if I need a transit...,594
1,Do I need a visa to transit (or layover) in th...,585
2,Should my first trip be to the country which i...,331
3,Should I submit bank statements when applying ...,259
4,How much electronics and other valuables can I...,197
5,How does the Schengen 90/180 rule work?,192
6,Do I need a US visa to transit (or layover) th...,165
7,I have two passports/nationalities. How do I u...,146
8,Schengen Visa Refusal: Justification for the p...,111
9,Can I visit Schengen countries on a Type-D Sch...,96


### 2. Rozwiązanie za pomocą wywołania ciągu „zwykłych” metod i funkcji z pakietu *pandas*.


In [10]:
RelatedTab = PostLinks.RelatedPostId.to_frame('PostId')
RelatedTab = RelatedTab.groupby(['PostId']).size().to_frame('NumLinks')
Post = Posts[Posts.PostTypeId == 1]
df_pd_2 = (pd.merge(Post, RelatedTab, left_on='Id', right_on='PostId')
           [[ 'Title', 'NumLinks']])
df_pd_2 =df_pd_2.nlargest(10, columns='NumLinks').reset_index(drop = True)
df_pd_2

,Title,NumLinks
0,Is there a way to find out if I need a transit...,594
1,Do I need a visa to transit (or layover) in th...,585
2,Should my first trip be to the country which i...,331
3,Should I submit bank statements when applying ...,259
4,How much electronics and other valuables can I...,197
5,How does the Schengen 90/180 rule work?,192
6,Do I need a US visa to transit (or layover) th...,165
7,I have two passports/nationalities. How do I u...,146
8,Schengen Visa Refusal: Justification for the p...,111
9,Can I visit Schengen countries on a Type-D Sch...,96


### 3. Sprawdzenie tożsamości zwracanych wyników

In [11]:
df_pd_2.equals(df_sql_2)

True

Zadanie 3
---------

### 1. Rozwiązanie za pomocą wywołania *pandas.read_sql_query("""zapytanie SQL""")*

In [12]:
df_sql_3 = pd.read_sql_query("""
                    SELECT DISTINCT
                        Users.Id,
                        Users.DisplayName,
                        Users.Reputation,
                        Users.Age,
                        Users.Location
                    FROM (
                            SELECT
                                Name, UserID
                            FROM Badges
                            WHERE Name IN (
                                SELECT
                                    Name
                                FROM Badges
                                WHERE Class=1
                                GROUP BY Name
                                HAVING COUNT(*) BETWEEN 2 AND 10
                            )
                            AND Class=1
                        ) AS ValuableBadges
                    JOIN Users ON ValuableBadges.UserId=Users.Id
                    """, conn)
df_sql_3

,Id,DisplayName,Reputation,Age,Location
0,108,Ankur Banerjee,31273,27.0,"London, UK"
1,19,VMAtm,18556,33.0,"Tampa, FL, United States"
2,101,Mark Mayo,121667,37.0,"Sydney, New South Wales, Australia"
3,466,iHaveacomputer,8360,NaN,Down underer
4,793,mindcorrosive,10531,32.0,Bulgaria
5,693,RoflcoptrException,33300,NaN,None
6,6669,Relaxed,69405,NaN,None
7,1737,Gayot Fow,70237,NaN,"London, United Kingdom"
8,39065,Pont,1004,NaN,Austria
9,19400,phoog,34342,50.0,"New York, NY"


### 2. Rozwiązanie za pomocą wywołania ciągu „zwykłych” metod i funkcji z pakietu *pandas*.


In [13]:
Name = (Badges[Badges.Class == 1].groupby('Name').filter(lambda g: len(g) >= 2)
           .groupby('Name').filter(lambda g: len(g) <= 10).groupby('Name').size().reset_index())
ValuableBadges = Badges[Badges.Name.isin(list(Name.Name))]
ValuableBadges = ValuableBadges[ValuableBadges.Class == 1][[ 'Name', 'UserId' ]]
df_pd_3 = ((pd.merge(ValuableBadges, Users, left_on='UserId', right_on='Id'))
            [['Id', 'DisplayName','Reputation','Age','Location']])
df_pd_3 = df_pd_3.drop_duplicates(subset = ['Id', 'DisplayName','Reputation','Age','Location'])
df_pd_3.reset_index(drop = True, inplace=True)
df_pd_3

,Id,DisplayName,Reputation,Age,Location
0,108,Ankur Banerjee,31273,27.0,"London, UK"
1,19,VMAtm,18556,33.0,"Tampa, FL, United States"
2,101,Mark Mayo,121667,37.0,"Sydney, New South Wales, Australia"
3,466,iHaveacomputer,8360,NaN,Down underer
4,793,mindcorrosive,10531,32.0,Bulgaria
5,693,RoflcoptrException,33300,NaN,NaN
6,6669,Relaxed,69405,NaN,NaN
7,1737,Gayot Fow,70237,NaN,"London, United Kingdom"
8,39065,Pont,1004,NaN,Austria
9,19400,phoog,34342,50.0,"New York, NY"


### 3. Sprawdzenie tożsamości zwracanych wyników

In [14]:
df_pd_3.equals(df_sql_3)

True